In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np

2023-04-16 12:30:52.885472: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 12:30:52.938139: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 12:30:52.938821: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 12:30:53.803701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df_t = pd.read_csv('temperature.csv',index_col=0)
df_h = pd.read_csv('humidity.csv',index_col=0)
df_p = pd.read_csv('pressure.csv',index_col=0)
df_w = pd.read_csv('wind_speed.csv',index_col=0)
df_h.columns

Index(['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles',
       'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque', 'Denver',
       'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis',
       'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta',
       'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh',
       'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston',
       'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya',
       'Jerusalem'],
      dtype='object')

In [3]:
def normalize(data):
    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    return (data - data_mean) / data_std

In [4]:
city = 'San Diego'
tempog = df_t[city]
temp = df_t[city].rename("temperature").to_frame(name='temperature')
humid = df_h[city].rename("humidity").to_frame(name='humidity')
press = df_p[city].rename("presure").to_frame(name='presure')
wind = df_w[city].rename("wind").to_frame(name='wind')


features = pd.concat([temp, press, humid, wind], axis=1)
# features.index = time
features = features.dropna()
features

featuresog = features
featuresog

,temperature,presure,humidity,wind
datetime,,,,
2012-10-01 13:00:00,291.530000,1013.0,82.0,0.0
2012-10-01 14:00:00,291.533501,1013.0,81.0,0.0
2012-10-01 15:00:00,291.543355,1013.0,81.0,0.0
2012-10-01 16:00:00,291.553209,1013.0,81.0,0.0
2012-10-01 17:00:00,291.563063,1013.0,80.0,0.0
...,...,...,...,...
2017-11-29 20:00:00,292.150000,1017.0,72.0,2.0
2017-11-29 21:00:00,292.740000,1017.0,72.0,1.0
2017-11-29 22:00:00,292.580000,1016.0,68.0,2.0


In [5]:
training_size = int ( 0.8 * features.shape[0])  

dataset=features.values
data_mean = dataset[:training_size].mean(axis=0)
data_std = dataset[:training_size].std(axis=0)
dataset = (dataset-data_mean)/data_std

dataset

array([[ 0.23627539, -0.58110974,  0.73378908, -1.15167403],
       [ 0.23685282, -0.58110974,  0.68310075, -1.15167403],
       [ 0.23847809, -0.58110974,  0.68310075, -1.15167403],
       ...,
       [ 0.40945551, -0.24393192,  0.02415242,  0.24995504],
       [ 0.41440351, -0.24393192, -0.22928924,  0.24995504],
       [ 0.21483404, -0.13153931,  0.22690575,  0.24995504]])

In [6]:
features = normalize(features.values)
features = pd.DataFrame(features)
features

,0,1,2,3
0,0.222888,-0.528665,0.731908,-1.181374
1,0.223482,-0.528665,0.680392,-1.181374
2,0.225155,-0.528665,0.680392,-1.181374
3,0.226829,-0.528665,0.680392,-1.181374
4,0.228502,-0.528665,0.628876,-1.181374
...,...,...,...,...
44894,0.328173,-0.061829,0.216750,0.162904
44895,0.428364,-0.061829,0.216750,-0.509235
44896,0.401194,-0.178538,0.010687,0.162904
44897,0.406288,-0.178538,-0.246892,0.162904


In [7]:
past_history = 48
future_target = 48
STEP = 1

In [8]:
# def multivariate_data(dataset, target, start_index, end_index, history_size,
#                       target_size, step, single_step=False):
#   data = []
#   labels = []

#   start_index = start_index + history_size
#   if end_index is None:
#     end_index = len(dataset) - target_size

#   for i in range(start_index, end_index):
#     indices = range(i-history_size, i, step)
#     data.append(dataset[indices])

#     if single_step:
#       labels.append(target[i+target_size])
#     else:
#       labels.append(target[i:i+target_size])

#   return np.array(data), np.array(labels)

In [9]:
# x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
#                                                  training_size, past_history,
#                                                  future_target, STEP)
# x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
#                                              training_size, None, past_history,
#                                              future_target, STEP)

In [10]:
# print ('Single window of past history : {}'.format(x_train_multi[0].shape))
# print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [11]:
# ighnore below
dataset = featuresog.values
dataset = dataset.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
#print('dataset.shape', dataset.shape)
num_of_features = len(features.columns)
print('Number of features', num_of_features)


Number of features 4


In [12]:
epochs=5
lstm_layers = 128
look_back = 24*3
num_of_features=4
batch_size=128

train_size_percent = 0.80
pred_col = featuresog.columns.get_loc("temperature")
print(pred_col)

# function to split the data
def create_dataset(dataset, pred_col, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), :]
    dataX.append(a)
    dataY.append(dataset[i + look_back, pred_col])
  return np.array(dataX), np.array(dataY)


train_size = int(len(dataset) * train_size_percent)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

trainX, trainY = create_dataset(train, pred_col, look_back=look_back)
testX, testY = create_dataset(test, pred_col, look_back=look_back)

model = Sequential()
model.add(LSTM(lstm_layers, input_shape=(look_back,num_of_features)))
model.add(Dense(future_target))
model.compile(loss='mean_squared_error', optimizer='adam')
history= model.fit(trainX, trainY,validation_split=0.30, epochs=epochs, batch_size=batch_size,shuffle=False)

0


2023-04-16 12:30:55.350145: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/5


2023-04-16 12:30:55.619171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 12:30:55.620796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 12:30:55.622110: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

196/197 [============================>.] - ETA: 0s - loss: 0.0212

2023-04-16 12:31:08.489064: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 12:31:08.490853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 12:31:08.492226: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

197/197 [==============================] - 15s 66ms/step - loss: 0.0212 - val_loss: 0.0218
Epoch 2/5
197/197 [==============================] - 12s 63ms/step - loss: 0.0086 - val_loss: 0.0176
Epoch 3/5
197/197 [==============================] - 12s 63ms/step - loss: 0.0045 - val_loss: 0.0116
Epoch 4/5
197/197 [==============================] - 12s 63ms/step - loss: 0.0039 - val_loss: 0.0157
Epoch 5/5
197/197 [==============================] - 13s 64ms/step - loss: 0.0038 - val_loss: 0.0166


In [13]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

   1/1121 [..............................] - ETA: 7:36

2023-04-16 12:32:00.826587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 12:32:00.828310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 12:32:00.829685: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

279/279 [==============================] - 3s 12ms/step


In [18]:
pd.DataFrame(trainPredict)
pd.DataFrame(trainX)

ValueError: Must pass 2-d input. shape=(35846, 72, 4)

In [ ]:
# Get something which has as many features as dataset
testPredict_extended = np.zeros((len(testPredict),num_of_features))
# Put the predictions there
testPredict_extended[:,pred_col] = testPredict[:,0]
# Inverse transform it and select the pred_col column.
testPredict = scaler.inverse_transform(testPredict_extended)[:,pred_col]

In [ ]:
import math

trainY_extended = np.zeros((len(trainY),num_of_features))
trainY_extended[:,pred_col]=trainY
trainY = scaler.inverse_transform(trainY_extended)[:,pred_col]

testY_extended = np.zeros((len(testY),num_of_features))
testY_extended[:,pred_col]=testY
testY = scaler.inverse_transform(testY_extended)[:,pred_col]

plt.plot(trainY, label = "line 1")
print(trainPredict)
print(trainY)
# plt.plot(range(0,len(trainPredict)),trainPredict, label = "line 2")
plt.legend()
plt.show()

plt.plot(trainPredict, label = "line 1")
# plt.plot(range(0,len(trainPredict)),trainPredict, label = "line 2")
plt.legend()
plt.show()

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
# for x, y in val_data_multi.take(5):
#   multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

for x, y in val_data_multi.take(5):
  multi_step_plot(x[0], y[0], model.predict(x)[0])